<a href="https://colab.research.google.com/github/Vitalyastiy/ad-hoc/blob/main/%D0%9F%D0%BE%D0%B8%D1%81%D0%BA%20%D0%B2%D0%B7%D0%B0%D0%B8%D0%BC%D0%BE%D1%81%D0%B2%D1%8F%D0%B7%D0%B8%2B%D0%BA%D0%BE%D0%BD%D0%BD%D0%B5%D0%BA%D1%82%D0%BE%D1%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подключаем библиотеки и загружаем и подготавливаем датасеты

In [ ]:
import py7zr
import pandas as pd
import pandas as pd
from datetime import date, timedelta
import win32com.client as win32
import schedule
import time
import teradatasql
import datetime
import seaborn as sns
import phik
from phik.report import plot_correlation_matrix
from phik import report

In [ ]:
user="vitaly.**"
password="***"
query = '''select * from *****.vf_ad_hoc_finance_org'''

In [ ]:
with teradatasql.connect(host="********.ru", user=user, password=password,logmech="ldap", tmode="TERA" ) as connect:
    df = pd.read_sql(query, connect)
df

In [ ]:
df.info()

In [ ]:
archive = py7zr.SevenZipFile('2022.7z', mode='r')
archive.extractall()
archive.close()

In [ ]:
df1 = pd.read_csv("МФО_01_12_2021 - 27_11_2022.csv")
df1

In [ ]:
input = df1['month']
qw=[]
# format
format = '%m_%Y'
for i in input:
    
    # convert from string format to datetime format
    # and get the date
    qw.append(datetime.datetime.strptime(i, format).date())

In [ ]:
df1['Month'] = qw
df1

In [ ]:
df1.isnull().sum() #.sum () - в датафрейме количетсво незаполненных ячеек - 0

msisdn       0
month        0
kolvo_mfo    0
kolvo_sms    0
Month        0
dtype: int64

In [ ]:
#Распределение по количеству
df1.groupby(['Month']).agg({'msisdn': ['count']}).reset_index().plot(kind='bar', x = 'Month', title= 'Распределение смс от МФО', color = 'red')



In [ ]:
dfg=df1.groupby(['Month'])[['kolvo_sms', 'msisdn', 'kolvo_mfo']].agg(['mean'])
dfg

In [ ]:
g = sns.lmplot(
    data=dfg,
    x="kolvo_sms", y="kolvo_mfo",
    height=5
)

In [ ]:
df['create_date_n']=df['create_date'].apply(lambda x: x.replace(day=1))
df

In [ ]:
df['create_date_1']=df['create_date_n'] - timedelta(1)

In [ ]:
df['create_date_1']=df['create_date_1'].apply(lambda x: x.replace(day=1))
df

In [ ]:
df.groupby(['create_date_n'])[['nps']].agg(['mean']).plot(kind='bar')


объединение данных


In [ ]:
# 1 без условий, по номеру
df_00 = pd.merge(df, df1, how='inner', on=['msisdn'])
df_00

In [ ]:
def f (row):
    if row ['kolvo_sms']>11 :
       val = 'yes' 
    else:
        val = 'no'
    return val
df_00['more then 11'] = df_00.apply (f, axis=1)
df_00['more then 11'].value_counts()

d_2345=df_00.groupby(['more then 11'])[['kolvo_mfo', 'kolvo_sms', 'nps']].agg([ 'mean', 'count'])
d_2345.columns = ['_'.join(col).strip() for col in d_2345.columns.values]
d_2345

In [ ]:
d00=df_00.groupby(['Month'])[['kolvo_sms', 'nps', 'kolvo_mfo']].agg(['mean'])
d00.corr()

In [ ]:
g0 = sns.lmplot(
    data=d00,
    x="nps", y="kolvo_sms",
    height=5
)
g1 = sns.lmplot(
    data=d00,
    x="nps", y="kolvo_mfo",
    height=5
)

Объеденение с уловиями месяц = месяц, месяц  = месяц - 1

In [ ]:
df_0 = pd.merge(df, df1, how='inner', left_on=['msisdn', 'create_date_n'] \
, right_on = ['msisdn','Month'])
df_0= df_0[['create_date', 'create_date_n', 'Month', 'msisdn', 'nps', 'kolvo_mfo', 'kolvo_sms', 'age', 'gender', 'ltr' ]]
df_0

In [ ]:
d0=df_0.groupby(['Month'])[['kolvo_sms', 'nps', 'kolvo_mfo']].agg(['mean'])
d0.corr()


In [ ]:
g0 = sns.lmplot(
    data=d0,
    x="nps", y="kolvo_mfo",
    height=5
)

In [ ]:
d02=df_0.groupby(['kolvo_sms'])[['kolvo_mfo', 'nps', ]].agg(['mean', 'sum'])
d02

In [ ]:
g04 = sns.lmplot(
    data=d02,
    x="nps", y="kolvo_sms",
    height=5
)

In [ ]:
df_0['bins']=pd.qcut(df_0['kolvo_sms'], q=4)
df_0

In [ ]:
df_0['bins'].value_counts()

In [ ]:
df11 = df_0[df_0['kolvo_sms'] >=11]
df11

In [ ]:
d_11=df11.groupby(['kolvo_mfo'])[['kolvo_sms']].agg(['mean', 'sum', 'count'])
d_11.columns = ['_'.join(col).strip() for col in d_11.columns.values]
d_11['kolvo_sms_mean'].plot(color ='tab:red', ylabel='kolvo_sms' , title='Зависимость смс от МФО')



In [ ]:
d_2=df_0.groupby(['bins'])[['kolvo_mfo', 'kolvo_sms', 'nps']].agg([ 'mean', 'sum'])
d_2.columns = ['_'.join(col).strip() for col in d_2.columns.values]
d_2[['kolvo_mfo_mean', 'kolvo_mfo_sum', 'kolvo_sms_mean', 'kolvo_sms_sum','nps_mean']]

In [ ]:
sns.heatmap(d_2[['kolvo_mfo_mean', 'kolvo_mfo_sum', 'kolvo_sms_mean', 'kolvo_sms_sum','nps_mean']].corr())

In [ ]:
df_0

In [ ]:
df_0.drop('bins_2', axis= 1 , inplace= True ) 

In [ ]:
def f (row):
    if row ['kolvo_sms']>11 :
       val = 'yes' 
    else:
        val = 'no'
    return val
df_0['more then 11'] = df_0.apply (f, axis=1)
df_0['more then 11'].value_counts()


In [ ]:
d_23=df_0.groupby(['more then 11'])[['kolvo_mfo', 'kolvo_sms', 'nps']].agg([ 'mean', 'count'])
d_23.columns = ['_'.join(col).strip() for col in d_23.columns.values]

In [ ]:
d_23

-month


In [ ]:
df_1 = pd.merge(df, df1, how='inner', left_on=['msisdn', 'create_date_1'] \
, right_on = ['msisdn','Month'])
df_1 = df_1[['create_date', 'create_date_1', 'Month', 'msisdn', 'nps', 'kolvo_mfo', 'kolvo_sms', 'age', 'gender', 'ltr' ]]
df_1

In [ ]:
df_1

In [ ]:
df12 = df_1[df_0['kolvo_sms'] >=11]
df12

In [ ]:
def f (row):
    if row ['kolvo_sms']>11 :
       val = 'yes' 
    else:
        val = 'no'
    return val
df_1['more then 11'] = df_1.apply (f, axis=1)
df_1['more then 11'].value_counts()

no     1280
yes     458
Name: more then 11, dtype: int64

In [ ]:
d_234=df_1.groupby(['more then 11'])[['kolvo_mfo', 'kolvo_sms', 'nps']].agg([ 'mean', 'count'])
d_234.columns = ['_'.join(col).strip() for col in d_234.columns.values]
d_234

In [ ]:
d1=df_1.groupby(['Month'])[['kolvo_sms', 'kolvo_mfo', 'nps']].agg([ 'mean'])
print(d1.corr())
sns.heatmap(df_1[['kolvo_mfo','kolvo_sms','nps']].corr())

In [ ]:
g2 = sns.lmplot(
    data=d1,
    x="nps", y="kolvo_sms",
    height=5
)


In [ ]:
d2=df_1.groupby(['kolvo_sms'])[['kolvo_mfo', 'nps']].agg([ 'mean', 'count'])
d2.columns = ['_'.join(col).strip() for col in d2.columns.values]
d2
#df_1['bins'] = pd.cut(d2['kolvo_sms'],3)


In [ ]:
df_1['bins']=pd.qcut(df_1['kolvo_sms'], q=4)
df_1

In [ ]:
df_1['bins'].value_counts()

In [ ]:
d3=df_1.groupby(['bins'])[['kolvo_mfo', 'kolvo_sms', 'nps']].agg([ 'mean', 'sum'])
d3.columns = ['_'.join(col).strip() for col in d3.columns.values]
d3[['kolvo_mfo_mean', 'kolvo_mfo_sum', 'kolvo_sms_mean', 'kolvo_sms_sum','nps_mean']]


In [ ]:
d3[['kolvo_mfo_mean', 'kolvo_mfo_sum', 'kolvo_sms_mean', 'kolvo_sms_sum','nps_mean']].corr()

In [ ]:
sns.boxplot(x="ltr", y="kolvo_mfo",
          
            data=df_0)

In [ ]:
df_0

In [ ]:
sns.scatterplot(data = df_0, x='kolvo_mfo' , y = 'kolvo_sms', hue = 'gender')

In [ ]:
sns.displot(data=df_0, x="ltr")
sns.displot(data=df_0, x="age")
